In [175]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.holtwinters import HoltWintersResults

import warnings
warnings.filterwarnings('ignore')

from datetime import datetime as dt
import numpy as np
import pandas as pd
import re

from sklearn.model_selection import ParameterGrid

# Read recipe inputs
#input_stacked = dataiku.Dataset("input_stacked")
input_stacked = dataiku.Dataset("Input_Partitioned")
input_df = input_stacked.get_dataframe()
input_df.head()

,PRODUCT,TYPE,INDICATION,STATE,STRENGTH,DATE,Y
0,VIVITROL,Demand,AD,OT15,All,2018-01-01 00:00:00+00:00,16467.971627
1,VIVITROL,Demand,AD,OT15,All,2020-07-01 00:00:00+00:00,27623.331733
2,VIVITROL,Demand,AD,OT15,All,2021-01-01 00:00:00+00:00,27820.958139
3,VIVITROL,Demand,AD,OT15,All,2021-10-01 00:00:00+00:00,14369.981105
4,VIVITROL,Demand,AD,OT15,All,2022-07-01 00:00:00+00:00,15923.695758


In [176]:
#PMV trouble-shooting
input_df = input_df[(input_df["PRODUCT"] == "MARKET") &
                    (input_df["TYPE"] == "TRx") &
                    (input_df["INDICATION"]  == 'atypical oral')
                    ]
input_df.head()

,PRODUCT,TYPE,INDICATION,STATE,STRENGTH,DATE,Y
323,MARKET,TRx,atypical oral,US,All,2016-08-01 04:00:00+00:00,4971630.0
324,MARKET,TRx,atypical oral,US,All,2021-03-01 05:00:00+00:00,5601079.0
325,MARKET,TRx,atypical oral,US,All,2021-08-01 04:00:00+00:00,5360062.0
326,MARKET,TRx,atypical oral,US,All,2021-09-01 04:00:00+00:00,5239156.0
327,MARKET,TRx,atypical oral,US,All,2022-11-01 04:00:00+00:00,5458567.0


In [177]:
#len(input_df)

In [178]:
def exp_smooth_tsd(df, config):
    
    df = df.reset_index(drop=True)
    print('########################')
    #print("##df##")
    #print(df)
    val_period = 2
    train_end = df['ds'][len(df['ds']) - (val_period + 1)]
    print("##train_end##")
    print(train_end)
    print("Timedelta",pd.Timedelta(df['ds'].diff()[1]).days )
    # determine frequency
    if pd.Timedelta(df["ds"].diff()[1]).days <= 31:
        df = df.set_index('ds').asfreq('MS')
        seasonal = 12
    else:
        df = df.set_index('ds').asfreq('QS')
        seasonal = 4
    
    print("seasonal", seasonal)
    #PMV edit: currently init_seasonal is initialized w/ use of estimated method
    init_seasonal = np.empty(seasonal)
    df_train = df[df.index < train_end].copy()
    print("##df_train##")
    print(df_train)
    model = ExponentialSmoothing(
        df_train['y'],
        trend=config['trend'],
        seasonal=config['seasonal'],
        damped_trend=config['damped_trend'],
        #seasonal_periods=seasonal,
        #PMV edit: below is used instead of initialization_method="known"
        initialization_method='estimated'
        #initialization_method='heuristic' #TROUBLESHOOTING
    ).fit()
    print("Model parameters: ", config)
    print(model.params_formatted)
    print(model.summary())
    
    proj_act = pd.DataFrame(model.fittedvalues, columns=["HWES"])
    print("#####proj act ######")
    print(proj_act)
    proj_forecast = pd.DataFrame(model.forecast(steps=val_period), columns=["HWES"])
    print("#####proj forecast ######")
    print(proj_forecast)
    proj = pd.concat([proj_act, proj_forecast])
    print("#####proj concatenated ######")
    print(proj)
    df = df.reset_index().merge(proj.reset_index().rename(columns={"index":"ds"}), how="outer", on="ds")
    print("#####merged df ######")
    print(df)
    
    df["error"] = (np.abs(df["HWES"] - df['y'])*100) / df['y']
    x = np.arange(0, len(df["error"].dropna()))
    print("x is", x)
    weights = x / np.linalg.norm(x)
    print("weights is", weights)

    df_error_dict = {
        'PRODUCT': df['PRODUCT'][0],
        'TYPE': df['TYPE'][0],
        'INDICATION': df['INDICATION'][0],
        'STATE': df['STATE'][0],
        'STRENGTH': df['STRENGTH'][0],
        'Ini_trend':model.params.get('initial_trend'),
        'Ini_season':model.params.get('initial_seasons'),
        'Ini_level':model.params.get('initial_level'),
        'Trend': config['trend'],
        'Seasonality': config['seasonal'],
        'Damped': config['damped_trend'],
        #'Damped': model.params.get('damped_trend'),
        'Alpha': model.params.get('smoothing_level'),
        'Beta': model.params.get('smoothing_trend'),
        'Gamma': model.params.get('smoothing_seasonal'),
        'Phi': model.params.get('damping_trend'), #PMV edit: 9/6/2023
        'Error_Percent_Weighted_Mean_Val': np.average(df["error"].dropna(), weights = weights),
        'BIC':model.bic
    }

    print(df_error_dict)

    print('########################')
    print("1st and 2nd fitting HWES & HWES2")
    model2 = ExponentialSmoothing(
        df_train['y'],
        trend=config['trend'],
        seasonal=config['seasonal'],
        damped_trend=config['damped_trend'],
        #initialization_method='heuristic', #TROUBLESHOOTING
        initialization_method='known',
        use_boxcox=model.params.get('use_boxcox'),
        initial_level=model.params.get('initial_level'),
        initial_trend=model.params.get('initial_trend'),
        initial_seasonal=model.params.get('initial_seasons')
    ).fit(
        smoothing_level=model.params.get('smoothing_level'),
        smoothing_trend=model.params.get('smoothing_trend'),
        smoothing_seasonal=model.params.get('smoothing_seasonal'),
        damping_trend = model.params.get('damping_trend'),
        remove_bias = model.params.get('remove_bias')
    )
        
    proj_act2 = pd.DataFrame(model2.fittedvalues, columns=["HWES2"])
    proj_forecast2 = pd.DataFrame(model2.forecast(steps=val_period), columns=["HWES2"])
    proj2 = pd.concat([proj_act2, proj_forecast2])
       
   
    print(df.reset_index().merge(proj2.reset_index().rename(columns={"index":"ds"}), how="outer", on="ds"))
    test_df = df.reset_index().merge(proj2.reset_index().rename(columns={"index":"ds"}), how="outer", on="ds")
    #print(df.dtypes)
    test_df['Diff'] = test_df['HWES']-test_df['HWES2']
    df_error_dict['avgdiff'] = np.average(test_df['Diff'].dropna())

    #print(model.fittedfcast)
    return df_error_dict

In [179]:
# fix date column
pd.to_datetime(input_df["DATE"])
input_df.rename(columns={"DATE": "ds", "Y": "y"}, inplace=True)
input_df['ds'] = input_df['ds'].apply(lambda x: x.replace(tzinfo=None))
# round is critical to handle changes in daylight savings from sql queries
input_df['ds'] = input_df['ds'].dt.round("D")

df_errors = []

params_grid = {
    'trend':['Add', 'Mul'],
    'seasonal':['Add', 'Mul'],
    'damped_trend':[True, False]
}
grid = ParameterGrid(params_grid)

In [180]:
for p in grid:
    group_errors = exp_smooth_tsd(df=input_df.sort_values(by=['ds']), config=p)
    df_errors.append(group_errors)

########################
##train_end##
2023-09-01 00:00:00
Timedelta 30
seasonal 12
##df_train##
           PRODUCT TYPE     INDICATION STATE STRENGTH          y
ds                                                              
2015-06-01  MARKET  TRx  atypical oral    US      All  4636230.0
2015-07-01  MARKET  TRx  atypical oral    US      All  4668132.0
2015-08-01  MARKET  TRx  atypical oral    US      All  4584888.0
2015-09-01  MARKET  TRx  atypical oral    US      All  4577077.0
2015-10-01  MARKET  TRx  atypical oral    US      All  4683518.0
2015-11-01  MARKET  TRx  atypical oral    US      All  4514043.0
2015-12-01  MARKET  TRx  atypical oral    US      All  4825701.0
2016-01-01  MARKET  TRx  atypical oral    US      All  4571076.0
2016-02-01  MARKET  TRx  atypical oral    US      All  4583107.0
2016-03-01  MARKET  TRx  atypical oral    US      All  4916342.0
2016-04-01  MARKET  TRx  atypical oral    US      All  4622135.0
2016-05-01  MARKET  TRx  atypical oral    US      All  479

In [181]:
#df_errors

In [182]:
# convert dict to dataframe
df_params_tsd = pd.DataFrame(df_errors)
df_params_tsd

,PRODUCT,TYPE,INDICATION,STATE,STRENGTH,Ini_trend,Ini_season,Ini_level,Trend,Seasonality,Damped,Alpha,Beta,Gamma,Phi,Error_Percent_Weighted_Mean_Val,BIC,avgdiff
0,MARKET,TRx,atypical oral,US,All,9106.934343,"[-55119.00260416688, -14312.992187499904, 1646...",4.670610e+06,Add,Add,True,0.181786,0.090893,0.094409,0.99,1.844228,2394.304151,2.341570e+03
1,MARKET,TRx,atypical oral,US,All,1.012051,"[-55119.00260416688, -14312.992187499904, 1646...",4.670610e+06,Mul,Add,True,0.181786,0.136339,0.094409,0.99,1.848841,2394.756327,-1.791003e+04
2,MARKET,TRx,atypical oral,US,All,9106.934343,"[0.988796720939342, 0.9966428865782541, 1.0345...",4.670610e+06,Add,Mul,True,0.181786,0.090893,0.094409,0.99,1.846266,2394.450036,0.000000e+00
3,MARKET,TRx,atypical oral,US,All,1.012051,"[0.988796720939342, 0.9966428865782541, 1.0345...",4.670610e+06,Mul,Mul,True,0.181786,0.075744,0.125879,0.99,1.841809,2394.374906,-3.056206e+04
4,MARKET,TRx,atypical oral,US,All,9106.934343,"[-55119.00260416688, -14312.992187499904, 1646...",4.670610e+06,Add,Add,False,0.181786,0.060595,0.094409,NaN,1.844267,2389.507161,4.656481e+03
5,MARKET,TRx,atypical oral,US,All,1.001950,"[-55119.00260416688, -14312.992187499904, 1646...",4.670610e+06,Mul,Add,False,0.181786,0.000100,0.094409,NaN,1.859146,2389.129287,-inf
6,MARKET,TRx,atypical oral,US,All,9106.934343,"[0.988796720939342, 0.9966428865782541, 1.0345...",4.670610e+06,Add,Mul,False,0.181786,0.060595,0.125879,NaN,1.844837,2389.601024,0.000000e+00
7,MARKET,TRx,atypical oral,US,All,1.001950,"[0.988796720939342, 0.9966428865782541, 1.0345...",4.670610e+06,Mul,Mul,False,0.181786,0.000100,0.125879,NaN,1.856428,2388.939094,0.000000e+00


In [183]:
#Code to catch the problem of : 
#statsmodels.tsa.holtwinters ExponentialSmoothing model using the same data and parameters 
#but returning two different results
#Example, Trend - Mul, Seasonal -Add parameters return erratic precitions when Damped Trend param is false
df = df_params_tsd
df = df[df['avgdiff'] != -np.inf]
df_params_tsd = df
df_params_tsd.shape


(7, 18)

In [184]:
# Select parameters with minimum error
#df_temp = pd.DataFrame(df_params_tsd.groupby(by=['PRODUCT', 'TYPE', 'INDICATION', 'STATE', 'STRENGTH'])["Error_Percent_Weighted_Mean_Val"].min()).reset_index()
df_temp = pd.DataFrame(df_params_tsd.groupby(by=['PRODUCT', 'TYPE', 'INDICATION', 'STATE', 'STRENGTH'])["BIC"].min()).reset_index()

In [185]:
df_temp.head()

,PRODUCT,TYPE,INDICATION,STATE,STRENGTH,BIC
0,MARKET,TRx,atypical oral,US,All,2388.939094


In [186]:
parameters_df = df_params_tsd.merge(df_temp, how="inner", on=['PRODUCT', 'TYPE', 'INDICATION', 'STATE', 'STRENGTH', "BIC"])
print(parameters_df)

  PRODUCT TYPE     INDICATION STATE STRENGTH  Ini_trend                                         Ini_season     Ini_level Trend Seasonality  Damped     Alpha    Beta     Gamma  Phi  Error_Percent_Weighted_Mean_Val          BIC  avgdiff
0  MARKET  TRx  atypical oral    US      All    1.00195  [0.988796720939342, 0.9966428865782541, 1.0345...  4.670610e+06   Mul         Mul   False  0.181786  0.0001  0.125879  NaN                         1.856428  2388.939094      0.0


In [187]:
'''#Trouble-shooting
s = [ -55119.00260417,  -14312.9921875 ,  164648.39322917,
       -143115.60677083,  137116.09114583,  -54149.2734375 ,
         57126.3828125 ,   26319.55989583, -300643.45052083,
        131789.84114583,  -75055.81510417,  125395.87239583] 

y = input_df.sort_values(by=['ds'])
y = y.set_index('ds').asfreq('MS')
#print(y)
forecast_length = 60
model = ExponentialSmoothing(
        y['y'],
        trend='Mul',
        seasonal='Add',
        damped_trend=False,
        initialization_method='known',
        initial_level=4670610.211111108,
        initial_trend=1.0019498382292253,
        initial_seasonal=s
).fit(
        smoothing_level=0.1817857142857143,
        smoothing_trend=0.000099999999999999,
        smoothing_seasonal=0.12587912087912087,
        damping_trend = np.nan)
print('Model SSE',model.sse)
proj_act = pd.DataFrame(model.fittedvalues, columns=["HWES"])
print(proj_act)'''

'#Trouble-shooting\ns = [ -55119.00260417,  -14312.9921875 ,  164648.39322917,\n       -143115.60677083,  137116.09114583,  -54149.2734375 ,\n         57126.3828125 ,   26319.55989583, -300643.45052083,\n        131789.84114583,  -75055.81510417,  125395.87239583] \n\ny = input_df.sort_values(by=[\'ds\'])\ny = y.set_index(\'ds\').asfreq(\'MS\')\n#print(y)\nforecast_length = 60\nmodel = ExponentialSmoothing(\n        y[\'y\'],\n        trend=\'Mul\',\n        seasonal=\'Add\',\n        damped_trend=False,\n        initialization_method=\'known\',\n        initial_level=4670610.211111108,\n        initial_trend=1.0019498382292253,\n        initial_seasonal=s\n).fit(\n        smoothing_level=0.1817857142857143,\n        smoothing_trend=0.000099999999999999,\n        smoothing_seasonal=0.12587912087912087,\n        damping_trend = np.nan)\nprint(\'Model SSE\',model.sse)\nproj_act = pd.DataFrame(model.fittedvalues, columns=["HWES"])\nprint(proj_act)'

In [188]:
#Trouble-shooting
s = [ -55119.00260417,  -14312.9921875 ,  164648.39322917,
       -143115.60677083,  137116.09114583,  -54149.2734375 ,
         57126.3828125 ,   26319.55989583, -300643.45052083,
        131789.84114583,  -75055.81510417,  125395.87239583]
#initial_seasonal_values = [float(x) for x in s.strip('[]').split()]

y = input_df.sort_values(by=['ds'])
y = y.set_index('ds').asfreq('MS')
#print(y)
forecast_length = 60
model = ExponentialSmoothing(
        y['y'],
        trend='Mul',
        seasonal='Add',
        damped_trend= True,
        initialization_method='known',
        initial_level=4670610.211111108,
        initial_trend=9106.934343434696,
        initial_seasonal=s
).fit(
        smoothing_level=0.181785714,
        smoothing_trend=0.060595238,
        smoothing_seasonal=0.125879121,
        damping_trend = np.nan)
print('Model SSE',model.sse)
proj_act = pd.DataFrame(model.fittedvalues, columns=["HWES"])
print(proj_act)

Model SSE nan
            HWES
ds              
2015-06-01   NaN
2015-07-01   NaN
2015-08-01   NaN
2015-09-01   NaN
2015-10-01   NaN
2015-11-01   NaN
2015-12-01   NaN
2016-01-01   NaN
2016-02-01   NaN
2016-03-01   NaN
2016-04-01   NaN
2016-05-01   NaN
2016-06-01   NaN
2016-07-01   NaN
2016-08-01   NaN
2016-09-01   NaN
2016-10-01   NaN
2016-11-01   NaN
2016-12-01   NaN
2017-01-01   NaN
2017-02-01   NaN
2017-03-01   NaN
2017-04-01   NaN
2017-05-01   NaN
2017-06-01   NaN
2017-07-01   NaN
2017-08-01   NaN
2017-09-01   NaN
2017-10-01   NaN
2017-11-01   NaN
2017-12-01   NaN
2018-01-01   NaN
2018-02-01   NaN
2018-03-01   NaN
2018-04-01   NaN
2018-05-01   NaN
2018-06-01   NaN
2018-07-01   NaN
2018-08-01   NaN
2018-09-01   NaN
2018-10-01   NaN
2018-11-01   NaN
2018-12-01   NaN
2019-01-01   NaN
2019-02-01   NaN
2019-03-01   NaN
2019-04-01   NaN
2019-05-01   NaN
2019-06-01   NaN
2019-07-01   NaN
2019-08-01   NaN
2019-09-01   NaN
2019-10-01   NaN
2019-11-01   NaN
2019-12-01   NaN
2020-01-01   NaN


In [189]:
#parameters_df['MODEL'] = 'HW'
# save the training date
#parameters_df['Forecast_date'] = dt.today().strftime('%Y-%m-%d')


# Write recipe outputs
##HW_Parameters = dataiku.Dataset("Holt_Winters_Model_Parameters")##
##HW_Parameters.write_with_schema(parameters_df)##

In [190]:
params_grid = {
    'trend':['Add', 'Mul'],
    'seasonal':['Add', 'Mul'],
    'damped_trend':[True, False]
}
grid = ParameterGrid(params_grid)
# Iterate over the parameter combinations
for params in grid:
    print(params)

{'damped_trend': True, 'seasonal': 'Add', 'trend': 'Add'}
{'damped_trend': True, 'seasonal': 'Add', 'trend': 'Mul'}
{'damped_trend': True, 'seasonal': 'Mul', 'trend': 'Add'}
{'damped_trend': True, 'seasonal': 'Mul', 'trend': 'Mul'}
{'damped_trend': False, 'seasonal': 'Add', 'trend': 'Add'}
{'damped_trend': False, 'seasonal': 'Add', 'trend': 'Mul'}
{'damped_trend': False, 'seasonal': 'Mul', 'trend': 'Add'}
{'damped_trend': False, 'seasonal': 'Mul', 'trend': 'Mul'}
